In [29]:
import nir
import torch
import numpy as np
from rockpool.nn.modules.torch.nir import from_nir, to_nir

In [2]:
!ls 

 2310_model_rework.ipynb	   lava_inference.ipynb
 analyze_graph.py		   lava_rnn.py
 Braille_inference.bak.ipynb	  'Norse inference.ipynb'
 Braille_inference.ipynb	   README.md
 Braille_inference_spinnaker2.py   rockpool_inference.ipynb
 braille.nir			   samna.log
 braille_subgraph.nir		   snntorch_debug_nirgraphs.ipynb
 collapsed.zip			   snntorch_debug.py
 data				   snntorch_test_export.py
 flatten_braille_graph.py	   snntorch_test_import.py
 generate_rnn_nir_graph.py


In [4]:
nir.read("braille.nir").edges

[('input', 'fc1'),
 ('lif1.w_rec', 'lif1.lif'),
 ('fc2', 'lif2'),
 ('lif1.lif', 'lif1.w_rec'),
 ('lif2', 'output'),
 ('fc1', 'lif1.lif'),
 ('lif1.lif', 'fc2')]

In [6]:
nirgraph = nir.read('braille.nir')
net = from_nir(nirgraph)
net

/home/jens/work/rockpool/rockpool/nn/modules/torch/linear_torch.py:111: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(weight) if weight is not None else None,
/home/jens/work/rockpool/rockpool/nn/modules/torch/torch_module.py:258: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  super().register_parameter(key, nn.Parameter(torch.tensor(value.data)))
/home/jens/work/rockpool/rockpool/nn/modules/torch/nir.py:82: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  dt=torch.min(torch.tensor(_to_tensor(node.tau_mem / (1+node.r)))).item(),


GraphExecutor(
  (fc1): LinearTorch()
  (fc2): LinearTorch()
  (input): Identity()
  (lif1_lif): LIFTorch()
  (lif1_w_rec): LinearTorch()
  (lif2): LIFTorch()
  (output): Identity()
)

In [11]:
### TEST DATA
test_data_path = "data/ds_test.pt"
ds_test = torch.load(test_data_path)

In [23]:
out = []
state = None
for t in ds_test[0][0]:
    z, state = net(t, state)
    out.append(z)
out = torch.stack(out)

In [31]:
import snntorch.functional as SF
### LOSS FUNCTION
loss_fn = SF.ce_count_loss()

def val_test_loop(dataset, batch_size, net, loss_fn, device, shuffle=True, saved_state_dict=None, label_probabilities=False, regularization=None):
  
  with torch.no_grad():
    if saved_state_dict != None:
        net.load_state_dict(saved_state_dict)
    net.eval()

    loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, drop_last=False)

    batch_loss = []
    batch_acc = []

    for data, labels in loader:
        data = data.to(device).swapaxes(1, 0)
        labels = labels.to(device)

        spk_out, hid_rec = net(data)

        # Validation loss
        if regularization != None:
            # L1 loss on spikes per neuron from the hidden layer
            reg_loss = regularization[0]*torch.mean(torch.sum(hid_rec, 0))
            # L2 loss on total number of spikes from the hidden layer
            reg_loss = reg_loss + regularization[1]*torch.mean(torch.sum(torch.sum(hid_rec, dim=0), dim=1)**2)
            loss_val = loss_fn(spk_out, labels) + reg_loss
        else:
            loss_val = loss_fn(spk_out, labels)

        batch_loss.append(loss_val.detach().cpu().item())

        # Accuracy
        act_total_out = torch.sum(spk_out, 0)  # sum over time
        _, neuron_max_act_total_out = torch.max(act_total_out, 1)  # argmax over output units to compare to labels
        batch_acc.extend((neuron_max_act_total_out == labels).detach().cpu().numpy()) # batch_acc.append(np.mean((neuron_max_act_total_out == labels).detach().cpu().numpy()))
    
    if label_probabilities:
        log_softmax_fn = nn.LogSoftmax(dim=-1)
        log_p_y = log_softmax_fn(act_total_out)
        return [np.mean(batch_loss), np.mean(batch_acc)], torch.exp(log_p_y)
    else:
        return [np.mean(batch_loss), np.mean(batch_acc)]

test_results = val_test_loop(ds_test, 16, net, loss_fn, "cpu")

print("Test accuracy: {}%".format(np.round(test_results[1]*100,2)))

Test accuracy: 14.29%
